# Crop polygon from mask image

## 1. From Contours

That can contains many contours in one single image

### 1.1. Create a single mask image

In [1]:
import os 
import cv2 
import numpy as np 
from tqdm import tqdm 
import shutil 
import matplotlib.pyplot as plt

In [2]:
ZOOM_LEVEL = '18'
PARENT_DIR = f'data/test/output'
INPUT_DIR = f'{PARENT_DIR}/{ZOOM_LEVEL}'
OUTPUT_DIR = f'{PARENT_DIR}/{ZOOM_LEVEL}_all/'
IMG_NAME = f'prediction_gan_{ZOOM_LEVEL}'

os.makedirs(os.path.join(OUTPUT_DIR), exist_ok=True)

In [3]:
# Move all tiles from seperate folders into a single folder
# for _, x in enumerate(os.listdir(INPUT_DIR)):
#     for y in os.listdir(os.path.join(INPUT_DIR, x)):
#         shutil.copy(os.path.join(INPUT_DIR, x, y),
#                     os.path.join(OUTPUT_DIR, f'{ZOOM_LEVEL}_{x}_{y}'))
        
# Merge all tile image into a single image
img_list = {}
for img_file in tqdm(sorted(os.listdir(OUTPUT_DIR)), total=len(os.listdir(OUTPUT_DIR))):
    zoom_level, x, y = img_file.replace('.png', '').strip().split('_')
    if x not in img_list:
        img_list[x] = []
    img = cv2.imread(os.path.join(OUTPUT_DIR, img_file))
    img_list[x].append(img)

img_vs = []
for k, values in img_list.items():
    img_v = cv2.vconcat(values)
    img_vs.append(img_v)

img = cv2.hconcat(img_vs)
cv2.imwrite(f"{PARENT_DIR}/{IMG_NAME}.png", img)

100%|██████████| 56/56 [00:00<00:00, 2042.25it/s]


True

### 1.2. Detect Contours

In [ ]:
def find_contours(img_path):
    image = cv2.imread(img_path, 0)
    _, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(image, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    image_list = []
    rect_list = []
    for idx, cnt in enumerate(contours):
        rect = cv2.boundingRect(cnt)
        x,y,w,h = rect
        rect_list.append(rect)
        with open(f'{img_path}.txt', 'a') as f:
            f.write(f'{idx}\t{rect}\n')
        bbox = image[y: y+h, x: x+w]
        h, w = bbox.shape
        if h < 256 and w < 256:
            img = np.zeros((256, 256))
            offset_x = 0 if 128 - w // 2 <= 0 else 128 - w // 2
            offset_y = 0 if 128 - h // 2 <= 0 else 128 - h // 2
            img[offset_y:  offset_y + h, offset_x: offset_x + w] = bbox
        if h > 256 or w > 256:
            img = cv2.resize(bbox, (256, 256))
        _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
        img = cv2.resize(img, (256, 256))
        image_list.append(img)

    return rect_list, image_list

In [ ]:
ZOOM_LEVEL = '19'
DIR1 = f'20230413/prediction_masks/prediction_masks_{ZOOM_LEVEL}_mask_test.png'
DIR2 = f'20230413/arcpy_masks/arcpy_masks_{ZOOM_LEVEL}_mask_test.png'
OUT_DIR1 = f'20230413/prediction_masks/cropped/{ZOOM_LEVEL}'
OUT_DIR2 = f'20230413/arcpy_masks/cropped/{ZOOM_LEVEL}'
OUT_DIR3 = f'20230413/predict2arcpy/test'

rect_list1, image_list1 = find_contours(DIR1)
rect_list2, image_list2 = find_contours(DIR2)

In [ ]:
for idx1, rect1 in enumerate(rect_list1):
    pt1 = (rect1[0], rect1[1])
    for idx2, rect2 in enumerate(rect_list2):
        pt2 = (rect2[0], rect2[1])
        if np.sqrt(abs(pt1[0] - pt2[0]) ** 2 + abs(pt1[1] - pt2[1]) ** 2) < 8:
            with open(f'20230413/mapping_test.txt', 'a') as f:
                f.write(f'{idx1}\t{idx2}\t{rect_list1[idx1]}\t{rect_list2[idx2]}\n')
            cv2.imwrite(os.path.join(OUT_DIR1, f"pred_mask_{ZOOM_LEVEL}_{idx1}.png"), image_list1[idx1])
            cv2.imwrite(os.path.join(OUT_DIR2, f"ideal_mask_{ZOOM_LEVEL}_{idx1}.png"), image_list2[idx2])

### 1.3. Merge two masks into one image

In [ ]:
for img_file in sorted(os.listdir(OUT_DIR1)):
    suffix = img_file.split('mask')[-1]
    idx = int(img_file.split('_')[-1].replace('.png', ''))
    img1 = cv2.imread(os.path.join(OUT_DIR1, img_file))
    img2 = cv2.imread(os.path.join(OUT_DIR2, f'ideal_mask{suffix}'))
    img = cv2.hconcat([img1, img2])
    cv2.imwrite(os.path.join(OUT_DIR3, f"mask_{idx:05}.png"), img)

## 2. From Tif file

* Create Tif file from Geojson file
* Crop each polygon based on .geojson and .tif files
* Each single image contains only one polygon shape

### 2.1. Simple creating data

In [30]:
import rasterio
import rasterio.mask
import geopandas as gpd


# Read roofs
gdf = gpd.read_file("data/prediction_shape_v2.geojson")  # Your roofs
gdf.head()

,id,class_label,geometry
0,0,BUILDING,"POLYGON ((126.87264 37.52606, 126.87264 37.526..."
1,1,BUILDING,"POLYGON ((126.86843 37.52606, 126.86838 37.526..."
2,2,BUILDING,"POLYGON ((126.87043 37.52606, 126.87036 37.526..."
3,3,BUILDING,"POLYGON ((126.86241 37.52606, 126.86233 37.526..."
4,4,BUILDING,"POLYGON ((126.86096 37.52644, 126.86096 37.526..."


In [31]:
df = gpd.GeoDataFrame(gdf, columns=['id', 'geometry'])
df = df.set_index('id')
print(df.head())

                                             geometry
id                                                   
0   POLYGON ((126.87264 37.52606, 126.87264 37.526...
1   POLYGON ((126.86843 37.52606, 126.86838 37.526...
2   POLYGON ((126.87043 37.52606, 126.87036 37.526...
3   POLYGON ((126.86241 37.52606, 126.86233 37.526...
4   POLYGON ((126.86096 37.52644, 126.86096 37.526...


In [34]:
# df.set_index("roof_id")
# roof = df.loc[2]  # Your roof id


# Open input raster and write masked (clipped) output raster
with rasterio.open("data/arcpy_shape_v2.tif") as src:
    for idx, row in df.iterrows():
        try:
            out_image, out_transform = rasterio.mask.mask(src, [row["geometry"]], crop=True, filled=True)
            out_meta = src.meta

            out_meta.update(
                {
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                }
            )

            with rasterio.open(f"data/outputs_arcpy/mask_{str(idx).zfill(5)}.tif", "w", **out_meta) as dest:
                dest.write(out_image)
        except Exception as e:
            print(f"Exception as {e}")

### 2.2. Get same polygon by calculating IOU score

In [4]:
import rasterio
import rasterio.mask
import geopandas as gpd

In [5]:
def read_data(df_path):
    df = gpd.read_file(df_path)
    df = gpd.GeoDataFrame(df, columns=['id', 'geometry'])
    df = df.set_index('id')
    return df

In [34]:
pp_df = read_data("20230417/train_v3/prediction_masks_train_v3.geojson")
arcpy_df = read_data("20230417/train_v3/arcpy_masks_train_v3.geojson")

In [35]:
print(len(pp_df.index))
print(len(arcpy_df.index))

1543
1532


In [36]:
def save_polygon(tif_path, idx1, idx2, row, output_path):
    with rasterio.open(tif_path) as src:
        try:
            out_image, out_transform = rasterio.mask.mask(src, [row["geometry"]], crop=True, filled=True)
            out_meta = src.meta

            out_meta.update(
                {
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                }
            )

            with rasterio.open(f"{output_path}/mask_{str(idx1).zfill(5)}_{str(idx2).zfill(5)}.tif", "w", **out_meta) as dest:
                dest.write(out_image)
        except Exception as e:
            print(f"Exception as {e}")

In [37]:
for idx1, pp_row in pp_df.iterrows():
    polygon1 = pp_row['geometry']
    for idx2, arcpy_row in arcpy_df.iterrows():
        polygon2 = arcpy_row['geometry']

        if polygon1.intersects(polygon2):
            iou = polygon1.intersection(polygon2).area / polygon1.union(polygon2).area 
            if iou > 0.9:
                with open("20230417/train_v3/train_mapping.txt", 'a') as fin:
                    fin.write(f"pp: idx1 = {idx1}\t arcpy: idx2 = {idx2}\n")
                save_polygon("20230417/train_v3/prediction_masks_18.tif", idx1, idx2, pp_row, "20230417/train_v3/prediction_masks/cropped/18")
                save_polygon("20230417/train_v3/arcpy_masks_18.tif", idx1, idx2, arcpy_row, "20230417/train_v3/arcpy_masks/cropped/18")


### 2.3. Merge two masks into one image

In [24]:
import os 
import cv2
import numpy as np
from tqdm import tqdm

In [38]:
A_dir = '20230417/train_v3/prediction_masks/cropped/18'
B_dir = '20230417/train_v3/arcpy_masks/cropped/18'
out_dir = '20230417/train_v3/train_v3'

for img_file in tqdm(os.listdir(A_dir), total=len(os.listdir(A_dir))):
    imageA = cv2.imread(os.path.join(A_dir, img_file), 0)
    imageB = cv2.imread(os.path.join(B_dir, img_file), 0)

    hA, wA = imageA.shape
    hB, wB = imageB.shape

    if (hA < 256 and wA < 256) and (hB < 256 and wB < 256):
        imgA = np.zeros((256, 256))
        imgB = np.zeros((256, 256))
        offset_x = 0 if 128 - wA // 2 < 0 else 128 - wA // 2
        offset_y = 0 if 128 - hA // 2 < 0 else 128 - hA // 2
        imgA[offset_y:offset_y + hA, offset_x:offset_x + wA] = imageA
        offset_x = 0 if 128 - wB // 2 < 0 else 128 - wB // 2
        offset_y = 0 if 128 - hB // 2 < 0 else 128 - hB // 2
        imgB[offset_y:offset_y + hB, offset_x:offset_x + wB] = imageB
    else:
        imgA = cv2.resize(imageA, (256, 256))
        imgB = cv2.resize(imageB, (256, 256))
    
    cv2.imwrite(os.path.join(out_dir, img_file.replace('mask', 'mask_3_').replace('.tif', '.png')), cv2.hconcat([imgA, imgB]))

100%|██████████| 976/976 [00:00<00:00, 1015.66it/s]


## 3. Create Data

### 3.1 Create train/val

In [4]:
import os
import cv2
from sklearn.model_selection import train_test_split 
import pandas as pd

img_list = {}

train_folders = ['20230417/train_v1/train_v1', '20230417/train_v2/train_v2', '20230417/train_v3/train_v3']

for train_folder in train_folders:
    for img_name in os.listdir(os.path.join(train_folder)):
        img_list[img_name] = cv2.imread(os.path.join(train_folder, img_name))

s = pd.Series(img_list)
train_data, val_data = [i.to_dict() for i in train_test_split(s, train_size=0.9)]

for k, value in train_data.items():
    cv2.imwrite(os.path.join('20230417/train', k), value)

for k, value in val_data.items():
    cv2.imwrite(os.path.join('20230417/val', k), value)

### 3.2. Create test


In [10]:
for img_file in os.listdir('20230417/test/prediction_masks/18_all'):
    imgA = cv2.imread(os.path.join('20230417/test/prediction_masks/18_all', img_file), 0)
    imgB = cv2.imread(os.path.join('20230417/test/arcpy_masks/18_all', img_file), 0)
    cv2.imwrite(os.path.join('20230417/test/test', img_file), cv2.hconcat([imgA, imgB]))

### 3.3. Cut out raster file based on geojson

In [7]:
# Read roof
arcpy_df = read_data("20230417/test/arcpy_masks_test_suwon.geojson")

for idx, row in arcpy_df.iterrows():
    # Open input raster and write masked (clipped) output raster
    with rasterio.open("20230417/test/rgb/rgb_18.tif") as src:
        out_image, out_transform = rasterio.mask.mask(src, [row["geometry"]], crop=True)
        out_meta = src.meta

        out_meta.update(
            {
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform,
            }
        )

        with rasterio.open(f"20230417/test/rgb/cropped/18/rgb_{str(idx).zfill(5)}.tif", "w", **out_meta) as dest:
            dest.write(out_image)